In [1]:
import os
import openai

In [2]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
from git import Repo

In [4]:
from pathlib import Path

In [5]:
pwd

'/Users/kristinm/Documents/-2023-openai-course/api-stuff/oa-blog-post-creator'

In [6]:
PATH_TO_BLOG_REPO = Path("/Users/kristinm/Documents/-2023-openai-course/api-stuff/oa-blog-post-creator/.git")

In [7]:
PATH_TO_BLOG = PATH_TO_BLOG_REPO.parent

In [8]:
PATH_TO_CONTENT = PATH_TO_BLOG/"content"

In [9]:
PATH_TO_CONTENT

PosixPath('/Users/kristinm/Documents/-2023-openai-course/api-stuff/oa-blog-post-creator/content')

In [10]:
PATH_TO_CONTENT.mkdir(exist_ok=True,parents=True)

In [11]:
def update_blog(commit_message='Updates blog'):
    # GitPython -- Repo locations
    repo = Repo(PATH_TO_BLOG_REPO)
    # git add .
    repo.git.add(all=True)
    # git commit -m "updates blog"
    repo.index.commit(commit_message)
    # git push
    origin = repo.remote(name='origin')
    origin.push()

In [12]:
random_text_string = "kdsaflkjiewoasvantluat"

In [13]:
with open(PATH_TO_BLOG/"index.html", 'w') as f:
    f.write(random_text_string)

In [14]:
update_blog()

In [15]:
import shutil
def create_new_blog(title,content,cover_image):
    cover_image= Path(cover_image)
    
    # auto file naming utility
    files = len(list(PATH_TO_CONTENT.glob("*.html")))
    new_title = f"{files+1}.html"
    path_to_new_content = PATH_TO_CONTENT/new_title
    
    shutil.copy(cover_image,PATH_TO_CONTENT)
    
    if not os.path.exists(path_to_new_content):
        # WRITE A NEW HTML FILE
        with open(path_to_new_content,"w") as f:
            f.write("<!DOCTYPE html>\n")
            f.write("<html>\n")
            f.write("<head>\n")
            f.write(f"<title> {title} </title>\n")
            f.write("</head>\n")
            
            f.write("<body>\n")
            f.write(f"<img src='{cover_image.name}' alt='Cover Image'> <br />\n")
            f.write(f"<h1> {title} </h1>")
            # OpenAI --->> Completion GPT -->> "hello\nblog post\n"
            f.write(content.replace("\n", "<br />\n"))
            f.write("</body>\n")
            f.write("</html>\n")
            print("Blog created")
            return path_to_new_content
    else:
        raise FileExistsError("File already exists, please check again your name! Aborting!")

In [26]:
path_to_new_content = create_new_blog('Test_title','thisisatest','templogo.png')

Blog created


In [17]:
# index.html --> Blog posts

In [22]:
from bs4 import BeautifulSoup as Soup

In [27]:
with open(PATH_TO_BLOG/"index.html") as index:
    soup = Soup(index.read())

In [28]:
str(soup)

'<!DOCTYPE html>\n\n<html>\n<head>\n<meta charset="utf-8"/>\n<meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n<title>Document</title>\n<meta content="" name="description"/>\n<meta content="width=device-width, initial-scale=1" name="viewport"/>\n<link href="" rel="stylesheet"/>\n</head>\n<body>\n<h1>My Blog Home Page</h1>\n<a href="index.html">Home</a>\n</body>\n</html>'

In [29]:
# checking duplicate links

In [30]:
def check_for_duplicate_links(path_to_new_content, links):
    urls = [str(link.get("href")) for link in links]
    content_path = str(Path(*path_to_new_content.parts[-2:]))
    # /pathtocontent/1.html check for overwriting
    return content_path in urls

In [33]:
def write_to_index(path_to_new_content):
    with open(PATH_TO_BLOG/"index.html") as index:
        soup = Soup(index.read())

    links = soup.find_all("a")
    last_link = links[-1]
    
    if check_for_duplicate_links(path_to_new_content, links):
        raise ValueError("Link does already exist!")
        
    link_to_new_blog = soup.new_tag("a", href=Path(*path_to_new_content.parts[-2:]))
    link_to_new_blog.string = path_to_new_content.name.split(".")[0]
    last_link.insert_after(link_to_new_blog)
    
    with open(PATH_TO_BLOG/"index.html", "w") as f:
        f.write(str(soup.prettify(formatter='html')))
        

In [34]:
write_to_index(path_to_new_content)

In [35]:
update_blog()